In [1]:
import warnings
warnings.filterwarnings('ignore')

# Research Question 1

## Does the style differ between different authors and projects?

a) Authors: See evaluation below

b) Projects: Not (yet) implemented [Dataloaders and Pairs are built up using authors only]

In [2]:
import pandas as pd
import numpy as np
import torch
import pytorch_lightning as L
from tqdm import tqdm

In [3]:
# Load Style_Model from trained model

import sys
sys.path.append('..')

from util.style_model import StyleModel

model = StyleModel()
model.load_state_dict(torch.load('../model/Style_Model.pt'))

<All keys matched successfully>

In [4]:
# load testing data
test_data = pd.read_pickle('../data/04c_Test_Set.pkl')

# use all data as test data to compare with baselines (very slow)
# test_data = pd.read_pickle('../data/03_Subset_Frequent_Committers.pkl')

### Calculate Embedding Centroid for each author

In [5]:
authors_centroids = {}

for group in tqdm(test_data.groupby('author_email')):
    author_embeddings = np.array([model(message).squeeze().detach().numpy() for message in group[1]['message']])
    authors_centroids[group[0]] = np.mean(author_embeddings, axis = 0)

100%|██████████| 7/7 [02:54<00:00, 24.96s/it]


### Calculate Cosine Distance between Author Centroids

In [6]:
from sklearn.metrics.pairwise import euclidean_distances

centroids_array = [value for value in authors_centroids.values()]

distances = pd.DataFrame(euclidean_distances(centroids_array, centroids_array))

for i in range(len(distances)):
    for j in range(i + 1):
        distances[j][i] = 0

distances.style.background_gradient(cmap='coolwarm').set_precision(3)

,0,1,2,3,4,5,6
0,0.000,0.288,0.245,0.525,0.227,0.357,0.306
1,0.000,0.000,0.304,0.521,0.306,0.388,0.321
2,0.000,0.000,0.000,0.515,0.192,0.260,0.234
3,0.000,0.000,0.000,0.000,0.525,0.566,0.513
4,0.000,0.000,0.000,0.000,0.000,0.285,0.211
5,0.000,0.000,0.000,0.000,0.000,0.000,0.199
6,0.000,0.000,0.000,0.000,0.000,0.000,0.000


### Half-Automated Evaluation of Style by K-Means Summary

Check K-Means Summary for Number of Authors and Projects per Cluster. If the number is low, it indicates that styles can be assigned to the different authors.

In [11]:
from sklearn.cluster import KMeans

embeddings = [model(message).squeeze().detach().numpy() for message in tqdm(test_data['message'])]

clustering = KMeans(7)
clustering.fit(embeddings)
clustering_prediction = clustering.predict(embeddings)

100%|██████████| 10684/10684 [04:23<00:00, 40.61it/s]


In [8]:
import sys
sys.path.append('..')

from util.clustering_eval import clustering_summary

k_means_summary = clustering_summary(clustering_prediction, clustering.n_clusters, test_data)
k_means_summary

There are 7 different authors.
There are 91 different projects.


,Number of Messages,Number of different Authors,Average number of commits per different Author,Most common Author,Number of different Projects,Average number of commits per different Project,Most common project
0,2673.0,7.0,381.857143,"('mark@mark-story.com', 830)",74.0,36.121622,"('saltstack_salt', 862)"
1,2163.0,7.0,309.000000,"('marijnh@gmail.com', 1026)",51.0,42.411765,"('codemirror_CodeMirror', 818)"
2,3242.0,7.0,463.142857,"('thatch45@gmail.com', 1007)",73.0,44.410959,"('saltstack_salt', 1531)"
3,449.0,4.0,112.250000,"('pedro@algarvio.me', 369)",7.0,64.142857,"('saltstack_salt', 435)"
4,873.0,6.0,145.500000,"('mike@silverorange.com', 844)",7.0,124.714286,"('silverorange_swat', 844)"
5,297.0,5.0,59.400000,"('thatch45@gmail.com', 153)",6.0,49.500000,"('saltstack_salt', 284)"
6,987.0,7.0,141.000000,"('mark@mark-story.com', 471)",34.0,29.029412,"('cakephp_cakephp', 458)"


In [9]:
k_means_summary.describe()

,Number of Messages,Number of different Authors,Average number of commits per different Author,Number of different Projects,Average number of commits per different Project
count,7.000000,7.000000,7.000000,7.0000,7.000000
mean,1526.285714,6.142857,230.307143,36.0000,55.761557
std,1158.543153,1.214986,153.670207,30.6159,32.329862
min,297.000000,4.000000,59.400000,6.0000,29.029412
25%,661.000000,5.500000,126.625000,7.0000,39.266693
50%,987.000000,7.000000,145.500000,34.0000,44.410959
75%,2418.000000,7.000000,345.428571,62.0000,56.821429
max,3242.000000,7.000000,463.142857,74.0000,124.714286


In [10]:
from util.clustering_eval import print_clustering_classes

print_clustering_classes(clustering_prediction, clustering.n_clusters, test_data, message_details=True)


________________ Class 0 ________________

___
1) 
API CHANGE Removed Member::init_db_fields(), its no longer needed due to the Member.PasswordEncyrption property changing from an ENUM to Varchar.

git-svn-id: svn://svn.silverstripe.com/silverstripe/open/modules/sapphire/branches/<I>@<I> <I>b<I>ca-7a2a-<I>-9d3b-<I>d<I>a<I>a9

- - - 
Committer: ingo@silverstripe.com
Project:   silverstripe_silverstripe-framework
___
2) 
BUGFIX: Make sure CMS link knows when its currently in the CMS (from r<I>)

git-svn-id: svn://svn.silverstripe.com/silverstripe/open/modules/sapphire/branches/<I>@<I> <I>b<I>ca-7a2a-<I>-9d3b-<I>d<I>a<I>a9

- - - 
Committer: ingo@silverstripe.com
Project:   silverstripe_silverstripe-framework
___
3) 
BackURL from request rather than URL param

No url mapping to $BackURL is defined anywhere in code,
so this condition would never be met. Setting BackURL
from form data makes more sense, and replicates the existing
logic from MemberLoginForm.

- - - 
Committer: ingo@silverst